In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
from tqdm import tqdm

session = requests.Session()

file = open('urls.csv')
data = list(csv.reader(file, delimiter=","))
file.close()

url = [item for sublist in data for item in sublist]

print(url)

price, enrollment, rating, num_ratings = [], [], [], []

for URL in tqdm(url):

    try:
        req = session.get(URL).text
        soup = BeautifulSoup(req, 'html.parser')
    except:
        continue
    try:
        meta_price = soup.find("meta", {"property": "udemy_com:price"})
        price.append(meta_price.get("content", None))
    except:
        continue
    try:
        enrollment.append(soup.find("div", {"class": "enrollment"}).text)
    except:
        continue
    try:
        rating_temp = soup.find("span", {"class": "star-rating--star-wrapper--5Fj0L star-rating--medium--3Lhzz star-rating--dark-background--4E2W3"})
        rating.append(rating_temp.find("span", {"class": "ud-heading-sm star-rating--rating-number--3l80q"}).text)
    except:
        continue
    try:
        num_ratings_temp = soup.find("div", {"class": "clp-lead__element-item clp-lead__element-item--row"})
        num_ratings_temp = num_ratings_temp.find("a")
        num_ratings_temp = num_ratings_temp.find_all("span")
        num_ratings.append(num_ratings_temp[-1].text)
    except:
        continue


In [ ]:
def remove_dollar_sign(price):
    for i in range(0, len(price)):
        price[i] = price[i][1:]

    return price


def remove_letters(enrollment):
    for i in range(0, len(enrollment)):
        enrollment[i] = enrollment[i].rstrip(" students")

    return enrollment


def remove_num_ratings(num_ratings):
    for i in range(0, len(num_ratings)):
        num_ratings[i] = num_ratings[i].lstrip("(")
        num_ratings[i] = num_ratings[i].rstrip(" ratings)")

    return num_ratings

In [ ]:
price = remove_dollar_sign(price)
enrollment = remove_letters(enrollment)
num_ratings = remove_num_ratings(num_ratings)

In [ ]:
rows = zip(price, enrollment, rating, num_ratings)
header_list = ('price', 'enrollment', 'rating', 'num_ratings')

with open('AWS_udemy_data.csv', "w") as f:
    writer = csv.writer(f)
    for row in rows:
        writer.writerow(row)